In [1]:
import numpy as np
import torch
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn

from util import *
from model import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
	torch.set_default_tensor_type('torch.cuda.FloatTensor')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# patent_company = load_binary('/app/data/new/patent_company_new.pkl')
# patent_term = load_binary('/app/data/new/patent_term_new.pkl')
# patent_company = torch.from_numpy(patent_company).to(torch.float)
# patent_term = torch.from_numpy(patent_term).to(torch.float)


# cpc = clamp(torch.matmul(patent_company.T, patent_company), 0, 1).to('cpu')
# save_binary(cpc, '/app/data/cpc_new.pkl')
# cpc = csr_matrix(cpc)
# adj_orig = cpc

# cpt = clamp(torch.matmul(patent_company.T, patent_term), 0, 1).to('cpu').T
# cpt.shape
# save_binary(cpt, '/app/data/cpt_new.pkl')
# cpt.shape
# cpt_tensor = cpt
# cpt = csr_matrix(cpt)

In [3]:
patent_company = load_binary('/app/data/new2/patent_company_new.pkl')
patent_term = load_binary('/app/data/new2/patent_term_new.pkl')
patent_company = torch.from_numpy(patent_company).to(torch.float)
patent_term = torch.from_numpy(patent_term).to(torch.float)


cpc = clamp(torch.matmul(patent_company.T, patent_company), 0, 1).to('cpu')
save_binary(cpc, '/app/data/new2/cpc_new.pkl')
cpc = csr_matrix(cpc)
adj_orig = cpc

cpt = clamp(torch.matmul(patent_company.T, patent_term), 0, 1).to('cpu').T
cpt.shape
save_binary(cpt, '/app/data/new2/cpt_new.pkl')
cpt.shape
cpt_tensor = cpt
cpt = csr_matrix(cpt)

In [4]:
adj_norm, adj_label, norm, weight_tensor = prepare_adj_for_training(cpc)

In [5]:
adj_norm

tensor(indices=tensor([[   0,    2,    4,  ...,   30,  190, 4737],
                       [   0,    0,    0,  ..., 4737, 4737, 4737]]),
       values=tensor([0.0119, 0.0047, 0.0029,  ..., 0.0310, 0.0822, 0.5000]),
       device='cpu', size=(4738, 4738), nnz=38948, layout=torch.sparse_coo)

In [6]:
bi_adj_norm, biadj_label, norm_bi, weight_tensor_bi = prepare_biadj_for_training(cpt)

/app/notebooks/util.py:161: RuntimeWarning: divide by zero encountered in power
  degree_mat_inv_sqrt_col = sp.diags(np.power(colsum, -0.5).flatten())


In [7]:
bi_adj_norm.shape

torch.Size([4738, 9572])

In [8]:
bi_adj_norm = bi_adj_norm.to(device)

In [9]:
feature = create_feature(adj_norm.shape[0])
feature = prepare_features_for_training(feature)

input_dim = feature.shape[1]
bipartite_dim = cpt.shape[1]

adj_norm = adj_norm.to(device)
adj_label = adj_label.to(device)
biadj_label = biadj_label.to(device).T
print('biadj_label shape: ', biadj_label.shape)
bi_adj_norm = bi_adj_norm.to(device).T
print('bi_adj_norm shape: ', bi_adj_norm.shape)


weight_tensor = weight_tensor.to(device)
weight_tensor_bi = weight_tensor_bi.to(device)
feature = feature.to(device)
dropout = 0.0
model = HeteroVGAE(input_dim, bipartite_dim, hidden1_dim=4, hidden2_dim=2, dropout=dropout)
model = model.to(device)
optimizer = Adam(model.parameters(), lr=0.01)

biadj_label shape:  torch.Size([4738, 9572])
bi_adj_norm shape:  torch.Size([9572, 4738])


In [10]:
# save_binary(adj_norm, '/app/data/new/adj_norm.pkl')
# save_binary(adj_label, '/app/data/new/adj_label.pkl')
# save_binary(norm, '/app/data/new/norm.pkl')
# save_binary(weight_tensor, '/app/data/new/weight_tensor.pkl')
# print('bi_adj_norm shape: ', bi_adj_norm.shape)
# save_binary(bi_adj_norm, '/app/data/new/bi_adj_norm.pkl')
# print('biadj_label shape: ', biadj_label.shape)
# save_binary(biadj_label, '/app/data/new/biadj_label.pkl')
# save_binary(norm_bi, '/app/data/new/norm_bi.pkl')
# save_binary(weight_tensor_bi, '/app/data/new/weight_tensor_bi.pkl')

save_binary(adj_norm, '/app/data/new2/adj_norm.pkl')
save_binary(adj_label, '/app/data/new2/adj_label.pkl')
save_binary(norm, '/app/data/new2/norm.pkl')
save_binary(weight_tensor, '/app/data/new2/weight_tensor.pkl')
print('bi_adj_norm shape: ', bi_adj_norm.shape)
save_binary(bi_adj_norm, '/app/data/new2/bi_adj_norm.pkl')
print('biadj_label shape: ', biadj_label.shape)
save_binary(biadj_label, '/app/data/new2/biadj_label.pkl')
save_binary(norm_bi, '/app/data/new2/norm_bi.pkl')
save_binary(weight_tensor_bi, '/app/data/new2/weight_tensor_bi.pkl')

bi_adj_norm shape:  torch.Size([9572, 4738])
biadj_label shape:  torch.Size([4738, 9572])


'Done'

In [11]:
def train(epoch):
    history = {'loss':[], 'acc':[], 'ap':[], 'roc':[]}

    model.train()
    for i in tqdm(range(epoch)):
        z_c, z_t, A_pred, Bi_pred = model(feature, adj_norm, bi_adj_norm)

        users_zc = z_c.clone()
        users_zc[16] = z_c[28]
        # print(users_zc[16])
        # print(z_c[16])
        # print(z_c[28])


        optimizer.zero_grad()

        loss = model.loss_function(norm, norm_bi, adj_label, biadj_label, A_pred, Bi_pred, weight_tensor, weight_tensor_bi, users_zc)

        loss.backward()
        optimizer.step()

        # embedding_company = z_c.cpu().detach().numpy()
        # embedding_term = z_t.cpu().detach().numpy()
        # # train_acc = get_acc(A_pred, adj_label)
        # roc_curr, ap_curr = get_scores(embedding_company, adj_orig, val_edges, val_edges_false)

        history['loss'].append(loss.item())
        # history['acc'].append(train_acc.item())
        # history['ap'].append(ap_curr)
        # history['roc'].append(roc_curr)

        # print("Epoch:", '%04d' % (i + 1), "train_loss=", "{:.5f}".format(loss.item()),
        #       "train_acc=", "{:.5f}".format(train_acc),
        #       "val_ap=", "{:.5f}".format(ap_curr), 
        #       "val_roc=", "{:.5f}".format(roc_curr))
        if i % 100 == 0:
            print(loss.item())
        embedding_company = z_c.to('cpu').detach()
        embedding_term = z_t.to('cpu').detach()
    return model, embedding_company, embedding_term, A_pred, history

In [12]:
num_train = 100
model, embedding_company, embedding_term, A_pred, history = train(num_train)
# torch.save(model.state_dict(), "/app/model/model-4.pt")

  0%|          | 0/100 [00:00<?, ?it/s]

  6%|▌         | 6/100 [00:00<00:03, 24.88it/s]

1932558.0


100%|██████████| 100/100 [00:03<00:00, 31.08it/s]


In [13]:
embedding_company.shape

torch.Size([4738, 2])

In [14]:
embedding_term.shape

torch.Size([9572, 2])